### Import packages

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
import re
import time
import pandas as pd
import joblib

In [5]:
newegg_link = "https://www.newegg.com/Laptops-Notebooks/SubCategory/ID-32"
state_filepath = r"E:\other\Epsilon_Ai\Final Project\final project\new_egg_linklast55.pkl"

# open chromedriver on your PC
def open_chromedriver():

    from selenium.webdriver import Chrome, ChromeOptions # Pictures are not displayed in page
                                                         # To download components quickly
        
    from selenium.webdriver.chrome.service import Service # to install The appropriate version for the user's browser
    from webdriver_manager.chrome import ChromeDriverManager
    
    global web
    chromedriver_installer = Service(ChromeDriverManager().install())
    
    chromedriver_options = ChromeOptions()
    prefs = {'profile.managed_default_content_settings.images' : 2} # Pictures are not displayed in page
    chromedriver_options.add_experimental_option('prefs', prefs)
    
    web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)
        
    return web

# to select maxmum number of laptops in page
def show_more():
    time.sleep(2)
    select_feat = web.find_element(by = 'class name', value = "list-tool-view").find_element(by="tag name", value="select")
    select_list = select_feat.text.split("\n")
    show_item = max(select_list) #maxmum number
    select_element = Select(select_feat)
    select_element.select_by_value(show_item)
    return show_item


# scroll down all the page to load all data of item                       
def scroll_down_all_the_way():
    
    from time import sleep 
    
    end = 0
    while end < web.execute_script('return document.body.scrollHeight'):
        start = end
        end = start + 700
        web.execute_script(f'window.scrollTo({start}, {end})')
        sleep(3)
        
# get all links of laptops            
def get_laptops_links():
    
    soup = BeautifulSoup(web.page_source, 'html.parser')
    item = soup.find_all("div", class_="item-cell")
    global links
    
    for i in item:
        try:
            link = i.a.attrs["href"]
            links.append(link)
            
        except:
            pass
    
    return links

# scrape data and append it in list of dictionary
def get_laptops_info(link):
    time.sleep(2)
    web.get(link)
    listing = {}
    listing["link"] = link
    try: 
        scroll_down_all_the_way()
    except:
        pass
    
    soup = BeautifulSoup(web.page_source, 'html.parser')
    
    listing["title"] = soup.find("div", class_="product-wrap").get_text(strip=True)
    listing["price"] = soup.find_all("li", class_="price-current")[-1].get_text(strip=True)
                
        
    tag = soup.find("div", class_="product-bullets")
    if tag:
        listing["feature"] = " ; ".join([info.get_text(strip=True) for info in tag.find_all("li")]) # Concate all features and spliting it by ";" 
        
    try:    
        specs = web.find_element(by="xpath", value="/html/body/div[8]/div[4]/div/div/div/div[2]/div[2]/div[1]/div[1]/div[6]/div[1]/div[2]").click()
    except:
        try:
            time.sleep(2)
            specs = web.find_element(by="xpath", value="/html/body/div[8]/div[4]/div/div/div/div[2]/div[2]/div[1]/div[1]/div[6]/div[1]/div[2]").click()
        except:
                time.sleep(3)
                specs = web.find_element(by="xpath", value="/html/body/div[11]/div/div[2]/div/a[2]").click()
    global keys
    global values
    keys = []
    values = []
    
     # to save data in dectionary       
    tag = soup.find("div", class_="tab-panes")
    if tag :
        tb = tag.find_all("tbody")
        
        for item in tb:
            key = [i.get_text(strip=True) for i in item.find_all("th")]
            value = [i.get_text(strip=True) for i in item.find_all("td")]

            keys.extend(key)
            values.extend(value)
            
        dict_value = dict(zip(keys, values))
        listing.update(dict_value)
    
    print(link)
    print(f"{len(listing)} - ", listing)
    print("-" *100, "\n\n")
    
    return listing


# to save all operation on PC
def save_state():
    
    import joblib
    
    global listings
    global state_filepath
    
    state = listings
    joblib.dump(state, state_filepath)

In [6]:
web = open_chromedriver()
web.get(newegg_link) # open link
soup = BeautifulSoup(web.page_source, 'html.parser') # send request

# pages number
pages = int(web.find_element(by="xpath", value="/html/body/div[8]/div[4]/section/div[1]/div/div[2]/div/div/div[2]/div[2]/div[1]/div[1]/div[4]/div/div/div[10]/button").text)
show_item = 0
show_item = show_more()
listings = []
links = []
all_links = []
select_list = []


for page in range(1, pages+1): # get all links
    new_egg_link = f"https://www.newegg.com/Laptops-Notebooks/SubCategory/ID-32/Page-{page}?PageSize={show_item}"
    try:
        web.get(new_egg_link)
        scroll_down_all_the_way()

    except:
        time.sleep(4) # check of the item link isnot laptop
        web.get(new_egg_link)
    links = get_laptops_links()
    all_links.extend(links)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Mora\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\Mora\AppData\Local\Temp/ipykernel_12076/1553296393.py:17: DeprecationWarning: use options instead of chrome_options
  web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)


In [3]:
new_links = joblib.load(r"E:\other\Epsilon_Ai\Final Project\final project\links.pkl")
new_links

['https://www.newegg.com/acer-sf514-55ta-77ww/p/N82E16834360115',
 'https://www.newegg.com/black-lenovo-20u5s0nq00-mainstream/p/1TS-000E-0X1A3',
 'https://www.newegg.com/blue-lenovo-82g4002qus/p/1TS-000E-0UNG3',
 'https://www.newegg.com/grey-lenovo-21cd000kus-content-creation/p/1TS-000E-0XTA7',
 'https://www.newegg.com/lenovo-20xf004nus-mainstream/p/1TS-000E-0WZH1',
 'https://www.newegg.com/intel-nuc-m15-lapbc710/p/N82E16834748018',
 'https://www.newegg.com/lenovo-82gw009aus-home-personal/p/1TS-000E-0VHH0',
 'https://www.newegg.com/indie-black-asus-vivobook-k513eq-nb74-home-personal/p/N82E16834235963',
 'https://www.newegg.com/helix-nuc-m15-lapbc710/p/N82E16834748017',
 'https://www.newegg.com/intel-nuc-m15-lapbc510-bbc510ecu7a01-work-business/p/N82E16834748016',
 'https://www.newegg.com/black-lenovo-thinkpad-e15-gen-2-20td00b7us-mainstream/p/2WC-000J-00TP8',
 'https://www.newegg.com/celestial-blue-asus-zenbook-ux482ear-db71t-home-personal/p/N82E16834236008',
 'https://www.newegg.com/s

In [11]:
links = new_links[0:10]

while links: # to git all links
    link = links.pop()# to search about link by link
    try:
        listing = get_laptops_info(link)
        listings.append(listing)
        
        
    except :
            print('timeout detected... cooling off...') # to make sure all informations is loaded
            time.sleep(60.0)
            listing = get_laptops_info(link)
            listings.append(listing)
            save_state()

https://www.newegg.com/intel-nuc-m15-lapbc510-bbc510ecu7a01-work-business/p/N82E16834748016
4 -  {'link': 'https://www.newegg.com/intel-nuc-m15-lapbc510-bbc510ecu7a01-work-business/p/N82E16834748016', 'title': 'Helix WorkPlex 1150N Premium 15.6" Touchscreen Notebook - Intel Core i5 11th Gen 1135G7 (2.40GHz), 8GB DDR4 4266MHz, 512GB PCIe 3.0 x4 NVMe SSD, Wi-Fi 6/ BT 5.1, Windows 10 Pro, 2 Years WarrantyBe the first to review this product...Ask Or Answer A QuestionShareSee more "mobile workstation"Check more best sellers of  "Business Laptops"Check more lowest price of  "Business Laptops"In stock.Ships from United States.Install Intel NUC Software Studio Service is Required.Learn MoreIntel Core i5 11th Gen512GB PCIe 3.0 x4 NVMe SSD8GB DDR4 4266 MHz RAM10 Point 1080p IPS TouchscreenCNC Anodized Aluminum Chassis2 x Thunderbolt 4 (Type-C)WiFi 6 / Bluetooth 5.1All Day Battery LifeAdaptive Usages: Wake on Approach, Walk Away LockWindows Hello Camera (authentication in Win 10 Pro through facia

In [12]:
import pandas as pd
import joblib


listings = joblib.load(r"E:\other\Epsilon_Ai\Final Project\final project\test\new_egg_link.pkl") # load data form PC
pd.options.display.max_columns = None # donot display all rows and columns of data

In [13]:
df = pd.DataFrame.from_records(listings)
df

,link,title,price,feature,Screen SizeScreen Size,Date First Available,Brand,Series,CPU Type,CPU SpeedCPU Speed,ResolutionResolution,Operating System,GPU/VPUGPU/VPU,Graphic TypeGraphic Type,MemoryMemory,WLANWLAN,WiFi Generation,BluetoothBluetooth,WebcamWebcam,Battery,Color,TouchscreenTouchscreen,Type,Operating SystemOperating System,Graphics CardGraphics Card,Communication,Tech,Other Features,Core Name,Number of CoresNumber of Cores,CPU FSB,CPU Support,Chipset,Display Type,LCD Features,SSD,Memory Speed,Memory Spec,Memory Type,Max Memory Supported,LANLAN,USBUSB,HDMI,Keyboard,Style,Battery Life,Weight,Model,Part Number,CPUCPU,Screen,StorageStorage,Video MemoryVideo Memory,Wide Screen SupportWide Screen Support,Video Memory,Optical Drive Type,Usage,HDD,Backlit Keyboard,Dimensions (W x D x H),Turbo FrequencyTurbo Frequency,CPU L2 CacheCPU L2 Cache,CPU L3 CacheCPU L3 Cache,Optical Drive Spec,Card Slot,Video Port,Other port,Docking Connector,Audio Ports,Audio,Speaker,Touchpad,Pointing Stick,Card Reader,AC Adapter,,HP 240 Black Bluetooth Mouse,HP 930 Creator Wireless Mouse,HP Bluetooth Travel Mouse,Add to Cart,Operating system,Memory layout (slots & size),Memory,Memory Note,Memory and storage,Optical drive,Cloud service,Storage type,Internal Storage,Hard drive (2nd),Processor cache,Processor core,Processor family,Processor,Display,Graphics (integrated),Battery type,Power supply type,Minimum dimensions (W x D x H),Webcam,Pointing device,Audio features,Ports,Network interface,Wireless,Expansion slots,Security management,Product color,Product design,Energy efficiency,Warranty,Optical DriveOptical Drive,Card slot,Package Content,HDD Interface,Package Type,Storage Spec
0,https://www.newegg.com/p/1TS-001A-00KC3,Visit IPC StoreAsus C434TA-DS588T Silver Touch...,$759.00,,"14.0""","March 19, 2021",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.newegg.com/p/0GA-02A0-00057,Visit Maxi-Aids Inc. StoreLow Vision NotebookB...,$6.95,Bold thick black lines spaced at .75 inches ; ...,NaN,"July 03, 2019",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.newegg.com/p/N82E16834845477,Visit Deus PC StoreLenovo ThinkPad X1 Carbon 7...,"$1,799.99",Intel Core i7 8th Gen 8665U (1.90GHz) ; 16 GB ...,"14.0""","January 30, 2022",Lenovo,ThinkPad,Intel Core i7 8th Gen,8665U (1.90GHz),2560 x 1440,Windows 10 Pro,UHD Graphics 620,Integrated Card,16 GB,IEEE 802.11ac,Wi-Fi 5,Yes,Yes,4-cell Lithium Ion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.newegg.com/p/1TS-0025-00178,Visit InBulk Electronics StoreMicrosoft Surfac...,"$1,999.95",Intel Core i7 1.90GHz ; 16GB Memory ; Touchscr...,"13.5""","February 05, 2020",Microsoft,NaN,Intel Core i7,1.90GHz,NaN,Windows 10 Home,NaN,Integrated Card,16GB,802.11ac,NaN,Yes,Yes,NaN,Platinum,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na